In [ ]:
# Multiprocessing is a computing technique in which a computer system uses two or more processors (or CPU cores) to execute multiple processes simultaneously, thereby improving system performance, speed, and reliability.

In [ ]:
import multiprocessing
import time

start = time.perf_counter()

def test_func1():
    print("do something")
    print("sleep for 1 sec")
    time.sleep(1)
    print("done with something")

p1 = multiprocessing.Process(target=test_func1)
p2 = multiprocessing.Process(target=test_func1)
    
p1.start()
p2.start()
    
p1.join()
p2.join()
    
end = time.perf_counter()
print(f"The program is finished in {round(end-start, 2)} seconds.")

The program is finished in 0.11 seconds.


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'test_func1' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/spawn.py", lin

## What this does:

1. **p1.start()** and **p2.start()** - Launch both processes simultaneously
2. **p1.join()** and **p2.join()** - Wait for both processes to complete before continuing
3. Since both processes run in parallel, the total time should be around **1 second** (not 2 seconds)

## Expected Output:
```
do something
sleep for 1 sec
do something
sleep for 1 sec
done with something
done with something

# Why `if __name__ == '__main__':` is Essential for Multiprocessing

## The Problem

On **Windows** and **macOS**, Python uses the `spawn` method to create new processes. This means each child process starts by importing the main module from scratch.

Without the `if __name__ == '__main__':` guard, here's what happens:

1. **Main script runs** → Creates Process 1 and Process 2
2. **Process 1 starts** → Imports the main module → Sees the process creation code → Creates MORE processes
3. **Process 2 starts** → Imports the main module → Sees the process creation code → Creates MORE processes
4. **Each new process** → Does the same thing → **Infinite loop of process spawning!**

## The Solution

```python
import multiprocessing

def worker_function():
    print("Working...")

if __name__ == '__main__':
    # This code ONLY runs in the main process
    # Child processes will NOT execute this block
    p1 = multiprocessing.Process(target=worker_function)
    p1.start()
    p1.join()
```

## How It Works

- When you run the script directly: `__name__` equals `'__main__'` → Code inside the guard runs
- When a child process imports the module: `__name__` equals the module name → Code inside the guard is **skipped**

This prevents child processes from spawning their own children.

## Platform Differences

| Platform | Default Method | Needs Guard? |
|----------|---------------|--------------|
| **Windows** | `spawn` | ✅ **Required** |
| **macOS** | `spawn` | ✅ **Required** |
| **Linux** | `fork` | ⚠️ Recommended (best practice) |

Even though Linux uses `fork` by default (which doesn't re-import the module), it's still best practice to always use the guard for cross-platform compatibility.

## Common Mistake

```python
# ❌ WRONG - Will cause infinite process spawning
import multiprocessing

def worker():
    print("Work")

p = multiprocessing.Process(target=worker)
p.start()
p.join()
```

```python
# ✅ CORRECT - Safe from infinite spawning
import multiprocessing

def worker():
    print("Work")

if __name__ == '__main__':
    p = multiprocessing.Process(target=worker)
    p.start()
    p.join()
```

## Remember

**Always define functions at module level (outside the guard) and put process creation code inside the guard!**